In [ ]:
# Enable ipywidgets under Google Colab (safe-no-op elsewhere)
from google.colab import output
output.enable_custom_widget_manager()

# 🧪 WFGY Diagnostic Notebook (v0.4)
**Minimal sandbox to measure ΔS and spot common reasoning failures**  
Runs in Colab or any Jupyter environment — no API key required.

## 📐 Core formula
$$\Delta S = 1 - \cos\theta\bigl(I, G\bigr) \qquad \bigl(I, G \in \mathbb R^{1536}\bigr)$$
*$I$ = current-step embedding,  $G$ = ground-truth / prompt embedding.*  
Higher ΔS ⇒ larger semantic drift ⇒ higher risk of collapse.

In [ ]:
!pip -q install sentence-transformers ipywidgets --upgrade

In [ ]:
from sentence_transformers import SentenceTransformer, util
from ipywidgets import Textarea, Button, VBox, Layout, Output
import sys

In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2')  # small & fast

In [ ]:
def delta_s(a_vec, b_vec):
    """ΔS = 1 – cosine_similarity"""
    return 1 - util.cos_sim(a_vec, b_vec).item()

def classify_failure(dS):
    if dS > 0.80:
        return ('No.1 Hallucination / Chunk Drift', 'Apply BBMC + boundary check')
    if dS > 0.60:
        return ('No.2 Interpretation Collapse', 'Insert λ_observe checkpoint')
    if dS > 0.40:
        return ('No.6 Logic Collapse', 'Trigger BBCR fallback')
    if dS > 0.25:
        return ('No.9 Entropy Collapse', 'Use BBAM attention modulation')
    return ('Stable (ΔS ≤ 0.25)', 'No action — pass')

In [ ]:
# --- Auto-detect widget support ---
try:
    import ipywidgets as _w; _ = _w.Button()
    _widget_ok = True
except Exception:
    _widget_ok = False

USE_WIDGETS = _widget_ok and ('google.colab' in sys.modules)

if USE_WIDGETS:
    demo_prompt = "### Product Review\nThe new phone has a crystal-clear display and fast charging…"
    demo_answer = "The capital of France is Paris. Also, your phone seems great!"

    p_box = Textarea(value=demo_prompt, description='Prompt', layout=Layout(width='100%', height='120px'))
    a_box = Textarea(value=demo_answer, description='Answer', layout=Layout(width='100%', height='120px'))
    out   = Output()

    def run_diag(_):
        out.clear_output()
        with out:
            dS = delta_s(model.encode(p_box.value, convert_to_tensor=True),
                         model.encode(a_box.value, convert_to_tensor=True))
            failure, fix = classify_failure(dS)
            print(f"ΔS : {dS:.3f}")
            print("Failure detected :", failure)
            print("Suggested fix    :", fix)

    btn = Button(description='Run diagnostic', button_style='primary')
    btn.on_click(run_diag)
    display(VBox([p_box, a_box, btn, out]))

else:
    print("Widget support disabled — fallback to CLI mode.\n")
    prompt = input("Prompt:\n")
    answer = input("\nAnswer:\n")
    dS = delta_s(model.encode(prompt, convert_to_tensor=True),
                 model.encode(answer, convert_to_tensor=True))
    failure, fix = classify_failure(dS)
    print("\nΔS :", f"{dS:.3f}")
    print("Failure detected :", failure)
    print("Suggested fix    :", fix)

---
### Next Steps  
* Explore the full **Problem Map 1.0** and RAG-focused **Problem Map 2.0**.  
* Read about **λ_observe, E_resonance, BBCR** in the WFGY 1.0 paper.  
* Fork this notebook, plug in your own embeddings, or extend the rules — MIT-licensed.  

> Repo → https://github.com/onestardao/WFGY